# Сложный пример генерации графа

В этом примере мы используем LLMClient для анализа более сложного историко-культурного текста.
Текст содержит различные сущности: географические объекты, исторических личностей, события, 
а также организации и предметы, упомянутые в тексте.

Мы не просто сгенерируем граф, но и применим к нему некоторую последующую логику:
- Отфильтруем узлы, которые не относятся к людям.
- Сохраним результирующий граф в локальный файл JSON.

Предполагается, что:
1. Модель 'mistral:7b' доступна.
2. Модуль с LLMClient доступен по пути относительно модуля.
3. Ollama настроен и работает.


In [1]:
import logging
import json
from graph_generator.core.llm_client import LLMClient

logging.basicConfig(level=logging.INFO)
llm_client = LLMClient(model="mistral:7b")

# Сложный текст: исторический отрывок с множеством взаимосвязанных элементов
complex_text = """
В начале XIX века, во время наполеоновских войн, Франция под предводительством Наполеона Бонапарта 
противостояла коалиции европейских держав: Великобритании, Австрии, Пруссии и России. В 1805 году состоялось 
крупное сражение при Аустерлице, где армия Наполеона одержала решающую победу над русско-австрийскими силами 
под командованием императора Александра I и императора Франца II. В этой битве также участвовал русский фельдмаршал Кутузов.

Спустя несколько лет, в 1812 году, Наполеон начал кампанию против России, двинувшись со своей Великой армией 
на Москву. Русские войска под командованием Кутузова применяли стратегию выжженной земли, отступая вглубь страны 
и не давая Наполеону решающего сражения. В итоге, после Бородинской битвы и оставления Москвы, французская армия 
была вынуждена отступать и понесла огромные потери.

В то же время британский дипломат лорд Каслри пытался укрепить международные союзы, поддерживая Пруссию и Австрию 
материально и дипломатически. В Лондоне активно обсуждались перспективы расширения колоний и усиления флота 
под командованием адмирала Нельсона, который, правда, погиб раньше — в 1805 году, но его победы на море 
служили примером для последующих поколений.

Кроме того, историки отмечают важную роль промышленной революции в Великобритании, которая позволила ей 
ускорять производство оружия и снаряжения. Торговые компании, такие как Ост-Индская компания, активно 
влияли на экономическую мощь британской короны.

Таким образом, на исторической сцене мы видим множество узлов: императоров, генералов, дипломатов, 
города, битвы, армии, флоты, торговые компании. Все они связаны сложной сетью взаимодействий и влияний.
"""

# Генерируем описание графа
graph_description = llm_client.generate_graph_description(complex_text)

# Выводим результат
print("Сгенерированная структура графа:")
print(json.dumps(graph_description, ensure_ascii=False, indent=4))

# Дополнительная логика: например, хотим отфильтровать только те узлы, которые относятся к людям.
# Предположим, что люди часто будут упомянуты в контексте имен. Сделаем простую эвристику:
human_nodes = {node: edges for node, edges in graph_description.items() if " " in node or node.endswith("I") or node.endswith("II")}

print("Фильтрованные узлы (предположительно люди):")
print(json.dumps(human_nodes, ensure_ascii=False, indent=4))

# Сохраним исходный граф в локальный файл
with open("complex_graph.json", "w", encoding="utf-8") as f:
    json.dump(graph_description, f, ensure_ascii=False, indent=4)


/home/h101/laba_works/nosignalx2k_works/graph_gen/ghgen/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Сгенерированная структура графа:
{
    "Наполеон Бонапарт": [
        "Франция",
        "Аустерлиц",
        "Коалиция европейских держав",
        "Александр I",
        "Франц II",
        "Кутузов"
    ],
    "Франция": [
        "Наполеон Бонапарт",
        "Аустерлиц",
        "Коалиция европейских держав",
        "Александр I",
        "Франц II",
        "Кутузов"
    ],
    "Коалиция европейских держав": [
        "Наполеон Бонапарт",
        "Аустерлиц",
        "Александр I",
        "Франц II",
        "Кутузов"
    ],
    "Аустерлиц": [
        "Наполеон Бонапарт",
        "Франция",
        "Коалиция европейских держав",
        "Александр I",
        "Франц II",
        "Кутузов"
    ],
    "Александр I": [
        "Наполеон Бонапарт",
        "Аустерлиц",
        "Коалиция европейских держав",
        "Франция",
        "Кутузов"
    ],
    "Франц II": [
        "Наполеон Бонапарт",
        "Аустерлиц",
        "Коалиция европейских держав",
        "Александр I",
    

# Пример генерации гиперграфа с дальнейшей обработкой

В этом примере мы:
1. Сгенерируем гиперграф для одного сложного текста.
2. Сгенерируем гиперграф для другого текста.
3. Объединим результаты, создав объединённый гиперграф, и применим логику преобразования.

Предполагается, что:
- Модель 'mistral:7b' доступна.
- Модуль LLMClient доступен по пути относительно модуля.
- Ollama настроен.

In [3]:
import logging
import json
from graph_generator.core.llm_client import LLMClient

logging.basicConfig(level=logging.INFO)
llm_client = LLMClient(model="mistral:7b")

text_1 = """
На празднование Нового года в древней столице собрались гости из разных регионов: 
Япония прислала делегацию самураев и торговцев, Китай — мудрецов и дипломатов, 
Россия — купцов и мастеровых, а Персия — поэтов и музыкантов. 
Все они участвовали в большом ритуале единения культур.
"""

text_2 = """
В северной провинции три племени — волхвы, охотники и кузнецы — сформировали союз 
для противостояния внешним угрозам. Их лидер, известный как Верховный шаман, 
собрал совет старейшин, чтобы обсудить распределение ресурсов и обязанности 
каждого племени в новом союзном образовании.
"""

# Генерация гиперграфов по двум разным текстам
hypergraph_1 = llm_client.generate_hypergraph_description(text_1)
hypergraph_2 = llm_client.generate_hypergraph_description(text_2)

print("Гиперграф 1:")
print(json.dumps(hypergraph_1, ensure_ascii=False, indent=4))

print("\nГиперграф 2:")
print(json.dumps(hypergraph_2, ensure_ascii=False, indent=4))

# Объединим два гиперграфа:
# Логика объединения: 
# - Добавим все гиперрёбра из обоих гиперграфов в один словарь.
# - Если в двух гиперграфах будут гиперрёбра с одинаковым именем, переименуем вторые.
merged_hypergraph = dict(hypergraph_1)  # Начинаем с копии первого

for hyperedge, nodes in hypergraph_2.items():
    new_hyperedge = hyperedge
    # Если имя уже занято, пытаемся добавить суффикс
    suffix = 2
    while new_hyperedge in merged_hypergraph:
        new_hyperedge = f"{hyperedge}_{suffix}"
        suffix += 1
    merged_hypergraph[new_hyperedge] = nodes

print("\nОбъединённый гиперграф:")
print(json.dumps(merged_hypergraph, ensure_ascii=False, indent=4))

# Применим логику очистки: удалим узлы, которые нигде не упомянуты.
# Для этого соберем все узлы из всех гиперрёбер, затем проверим, нет ли "висячих" узлов.
all_nodes = set()
for nodes in merged_hypergraph.values():
    for n in nodes:
        all_nodes.add(n)

# Предположим, что у нас есть какие-то узлы, добавленные вручную или ошибочно (их нет в примере, но логика показана)
# Если бы они были, мы могли бы их удалить — так как мы знаем, что узлы у нас имплицитно определяются гиперрёбрами,
# в данном случае все узлы уже связаны как минимум с одним гиперребром.

# Сохраним объединённый гиперграф
with open("merged_hypergraph.json", "w", encoding="utf-8") as f:
    json.dump(merged_hypergraph, f, ensure_ascii=False, indent=4)

print("\nСохранён объединённый гиперграф в файл: merged_hypergraph.json")


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:graph_generator.core.llm_client:Повторная попытка 2/3 через 2 секунды...
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:graph_generator.core.llm_client:Повторная попытка 2/3 через 2 секунды...
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:graph_generator.core.llm_client:Повторная попытка 3/3 через 2 секунды...
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Гиперграф 1:
{
    "Ритуал_Единения_Культур": [
        "гости",
        "самураи",
        "торговцы_Японии",
        "мудрецы_Китая",
        "дипломаты",
        "купцы_России",
        "мастеровые",
        "персийские_поэты",
        "музыканты"
    ],
    "Собрание_Гостей": [
        "гости",
        "самураи",
        "торговцы_Японии",
        "мудрецы_Китая",
        "дипломаты",
        "купцов_России",
        "мастеровых",
        "персийские_поэты",
        "музыкантов"
    ],
    "Делегация_Самураев": [
        "самураи"
    ],
    "Делегация_Торговцев_Японии": [
        "торговцы_Японии"
    ],
    "Делегация_Мудрецов_Китая": [
        "мудрецы_Китая",
        "дипломаты"
    ],
    "Делегация_Купцов_России": [
        "купцов_России"
    ],
    "Мастеровые": [
        "мастеровые"
    ],
    "Персийские_Поэты": [
        "персийские_поэты",
        "музыканты"
    ]
}

Гиперграф 2:
{
    "совет_старейшин": [
        "верховный_шаман"
    ],
    "племена": [
        "вол